In [1]:
import json
import hashlib
import os

import spacy

In [2]:
text = """
Bastien was born in New York.
Robert work as a Data Scientist.
Charles is the son of Robert. He was born in 1993. In 1993 Bastien went to the world in New York.
Charles graduated from EPFL in 2019. """

In [3]:
nlp = spacy.load('en_core_web_sm')

In [4]:
def split_by_sentence(text):
    doc = nlp(text)
    return [str(sent) for sent in doc.sents]
sents = split_by_sentence(text)

In [5]:
def encode_hash(obj):
    return hashlib.sha224(str(obj).encode()).hexdigest()

In [6]:
id2ent = {}
def detect_entities(sent):
    json_obj = []
    doc = nlp(sent)
    for ent in doc.ents:
        for ent2 in doc.ents:
            if ent != ent2:
                id2ent[encode_hash(ent)] = str(ent)
                json_obj.append({'sentence': sent,
                                'head': {
                                    'word': str(ent),
                                    'id': str(encode_hash(ent))
                                } ,
                                 'tail': {
                                     'word': str(ent2),
                                     'id': str(encode_hash(ent2))
                                 },
                                'relation': 'NA'})
    return json_obj

In [7]:
json_obj = []
for sent in sents:
    json_obj += detect_entities(sent)

In [8]:
id2ent

{'0b964d5916eaf00d0742cdd9acca7acd65bc2f8dc8de8ec1f843242c': 'New York',
 '0f489ae6101d4abec45305e100e1ad1ea2b511d37776af2624d23c24': 'Bastien',
 '1284904f6b4233d25240df203f571bb9f03e9b2f314952d06047cfb8': '1993',
 '30c6dad67c49781a68a4e2487bc15474e5efd2e03267e2e76c956c0f': '2019',
 '48837a787f07673545d9c610bcbcd8d46a2691a71966d856c197e69e': '\n',
 'a4d30cab6f9ff4305ffe143e2b68c10bfea759b90c4c78afafcc2e38': 'EPFL',
 'eb26be32bb78cb41db8ed8cda978941a2b01af786ca9c4de6fbbb2a3': 'Robert',
 'fbdc2183ea96e9796b68e99fbb4ed27922ed56758dfed1e436b1e8d1': 'Charles'}

In [9]:
directory = '../data/pred'
if not os.path.exists(directory):
    os.mkdir(directory)
with open('../data/pred/test.json', 'w') as f:
    f.write(json.dumps(json_obj))